In [ ]:
import numpy as np
import pickle, copy
import argparse
import re
import string
from nltk.corpus import stopwords
import pandas as pd
import random import shuffle

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from nltk.util import ngrams
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.utils.class_weight import compute_class_weight
from sklearn.semi_supervised import LabelSpreading

In [ ]:
nltk.download('punkt')

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("--task", required=True, help="USA or India")
parser.add_argument("--data", required=True, help="path of the directory containing data files")

args = vars(parser.parse_args())
data_dir = args["data"]
task_name = args["task"]

In [ ]:
with open(data_dir + '/train', 'rb') as F:
    train = pickle.load(F)

with open(data_dir + '/test', 'rb') as F:
    test = pickle.load(F)

with open(data_dir + '/EncodedDataFrameWithLabel', 'rb') as F:
    EncodedTweetDFLabel = pickle.load(F)

TweetInfoDF = pd.read_csv(data_dir + '/TweetInfoDF.csv')

In [ ]:
def remove_punctuations_and_numbers(String):
    L = []
    for s in tokenize(String):
        if(s not in string.punctuation)and not(s>="0" and s<="9")  and not(s=="…") and 'a'<=s and s<='z':
            L.append(s)
    return " ".join(L)

def tokenize(String):
    return nltk.word_tokenize(String)

def remove_stopwords(List):
    L = []
    for s in List:
        if(s not in stopwords.words("english") and (s not in hindi_stp) and (s not in hinglish_stp)):
            L.append(s)
    return L

def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

def strip_all_entities(text):
    entity_prefixes = ['@']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

In [ ]:
TweetInfoDFText = list(TweetInfoDF['text'])

EncodedTweetDFLabel['Tweet'] = list(TweetInfoDF['text'])
EncodedTweetDFLabel_without_label = EncodedTweetDFLabel[EncodedTweetDFLabel['Tag']==-1]
EncodedTweetDFLabel_without_label_INFO = copy.deepcopy(EncodedTweetDFLabel_without_label)

EncodedTweetDFLabel_without_label_INFO_cleaned_tweets = []

for i,j in EncodedTweetDFLabel_without_label_INFO.iterrows():
    tw = TweetInfoDFText[i].lower()
    tw = strip_links(strip_all_entities(tw))
    tw = remove_punctuations_and_numbers(tw)
    EncodedTweetDFLabel_without_label_INFO_cleaned_tweets.append(tw)


In [ ]:
if task_name=='India':
    EncodedTweetDFLabel_without_label_INFO_cleaned_tweets = list(set(EncodedTweetDFLabel_without_label_INFO_cleaned_tweets))
    shuffle(EncodedTweetDFLabel_without_label_INFO_cleaned_tweets)
    EncodedTweetDFLabel_without_label_INFO_cleaned_tweets = EncodedTweetDFLabel_without_label_INFO_cleaned_tweets[:60000]

EncodedTweetDFLabel_without_label_INFO_tag = [-1 for _ in EncodedTweetDFLabel_without_label_INFO_cleaned_tweets]

In [ ]:
cleaned_tweets = []

for i,j in train.iterrows():
    tw = TweetInfoDFText[i].lower()
    tw = strip_links(strip_all_entities(tw))
    tw = remove_punctuations_and_numbers(tw)
    cleaned_tweets.append(tw)

train_cleaned_tweets = copy.deepcopy(cleaned_tweets)

cleaned_tweets = []

for i,j in test.iterrows():
    tw = TweetInfoDFText[i].lower()
    tw = strip_links(strip_all_entities(tw))
    tw = remove_punctuations_and_numbers(tw)
    cleaned_tweets.append(tw)

test_cleaned_tweets = copy.deepcopy(cleaned_tweets)

cleaned_tweets = []
cleaned_tweets = test_cleaned_tweets
cleaned_tweets.extend(train_cleaned_tweets)

word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    stop_words='english',
    ngram_range=(1, 3),
    max_features=30000)
word_vectorizer.fit(cleaned_tweets)
dataset_word_features = word_vectorizer.transform(cleaned_tweets)
print(dataset_word_features.shape)

char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 4),
    max_features=30000)
char_vectorizer.fit(cleaned_tweets)
dataset_char_features = char_vectorizer.transform(cleaned_tweets)
print(dataset_char_features.shape)

dataset_features = hstack([dataset_char_features, dataset_word_features])

dataset_features = dataset_features.todense()

In [ ]:
X_test = dataset_features[:test.shape[0]]
X_train = dataset_features[test.shape[0]:]
y_test = list(test['Tag'])
y_train = list(train['Tag'])
y_train.extend(list(EncodedTweetDFLabel_without_label_INFO_tag))

print(X_train.shape, X_test.shape)

In [ ]:
clf = LabelSpreading('rbf')
clf.fit(X_train, y_train)
print(clf)

pred_y_test = clf.predict(X_test)
print(classification_report(y_test, pred_y_test))